A股（至少是指数）是否有周内效应呢？就是特定周几盈利高一些，让我们来验证一下吧。

In [1]:
import pandas as pd
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
from datetime import datetime
import json
import numpy as np
import trdb2py

import plotly.express as px

isStaticImg = True

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [2]:
paramsaip = trdb2py.trading2_pb2.AIPParams(
    money=10000,
    type=trdb2py.trading2_pb2.AIPTT_MONTHDAY,
    day=1,
)    

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

buyup = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ema.31'],
)

buydown = trdb2py.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['down'],
    strVals=['ema.31'],
)

lststart = [1, 2, 3, 4, 5]
lstend = [2, 3, 4, 5, 1]
lsttitle = ['周一', '周二', '周三', '周四', '周五']
lstparams = []
assets = ['jrj.510310']
for day in range(0, 4):
    for i in range(0, 5):
        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[lststart[i]],
        )

        sell0 = trdb2py.trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[trdb2py.nextWeekDay(lststart[i], day + 1)],
        )

        paramsbuy = trdb2py.trading2_pb2.BuyParams(
            perHandMoney=1,
        )

        paramssell = trdb2py.trading2_pb2.SellParams(
            perVolume=1,
        )
        
        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2Asset(assets[0]),
        )

        s0.buy.extend([buy0, buyup])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2Asset('jrj.510310')],
            startTs=0,
            endTs=-1,
            strategies=[s0],
            title='{}{}up'.format(day, i),
        ))
        
        s1 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2Asset(assets[0]),         
        )

        s1.buy.extend([buy0, buydown])
        s1.sell.extend([sell0])
        s1.paramsBuy.CopyFrom(paramsbuy)
        s1.paramsSell.CopyFrom(paramssell)
        s1.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2Asset('jrj.510310')],
            startTs=0,
            endTs=-1,
            strategies=[s1],
            title='{}{}down'.format(day, i),
        ))        
        
lstpnlaip = trdb2py.simTradings(trdb2cfg, lstparams)

# lstpnlaip

# # baseline    
# buy0 = trading2_pb2.CtrlCondition(
#     name='buyandhold',
# )

# paramsbuy = trading2_pb2.BuyParams(
#     perHandMoney=1,
# )

# paramsinit = trading2_pb2.InitParams(
#     money=10000,
# )

# pnl = libtrdb2.simTradingEx3(trdb2cfg, assets, 0, -1, [buy0], None, paramsbuy, None, paramsinit, None)
# lstpnlaip.append({'title': 'baseline', 'pnl': pnl})

dfaip = trdb2py.buildPNLReport(lstpnlaip)

In [3]:
dfaip[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

,title,maxDrawdown,maxDrawdownStart,maxDrawdownEnd,totalReturns,sharpe,annualizedReturns,annualizedVolatility,variance
37,33up,0.170116,2018-01-26,2019-01-17,3.096359,1.799663,0.277092,0.137299,0.382053
27,23up,0.166568,2015-11-09,2016-06-24,3.012993,1.922104,0.266073,0.122820,0.268228
28,24up,0.178435,2018-01-24,2019-01-14,2.661335,1.648368,0.219591,0.115018,0.242617
16,13up,0.169808,2015-11-09,2017-02-03,2.541792,1.596360,0.203790,0.108867,0.170168
35,32up,0.194929,2015-11-09,2016-06-24,2.506557,1.145043,0.199133,0.147709,0.216201
18,14up,0.128242,2018-01-23,2019-01-14,2.466925,1.624383,0.193895,0.100897,0.161025
8,04up,0.098497,2014-12-29,2015-02-13,2.220979,1.544400,0.161386,0.085073,0.100767
38,34up,0.274548,2017-11-22,2019-01-14,1.900047,0.661722,0.118966,0.134447,0.127844
25,22up,0.268510,2015-06-12,2016-06-24,1.721041,0.464909,0.095306,0.140470,0.069960
0,00down,0.177916,2015-06-30,2016-02-01,1.661136,0.790046,0.087387,0.072638,0.027998
